<a href="https://colab.research.google.com/github/idribeir0/machine-learning-dio/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de transfer learning em python realizado para a trilha de machine learning specialist da DIO. Utilizo a biblioteca tensorflow e também um dataset já existente para treinar a partir dele o meu modelo.

In [28]:
import tensorflow as tf
import tensorflow_datasets as tfds

carregando dataset

In [30]:
(ds_train, ds_val), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

redimensionando e normalizando as imagens

In [31]:
IMG_SIZE = 160

def format_example(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

ds_train = ds_train.map(format_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_val = ds_val.map(format_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)

BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_train = ds_train.cache().shuffle(1000).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

In [32]:
for image, label in ds_train.take(1):
    print("Image shape:", image.numpy().shape)
    print("Label:", label.numpy())


Image shape: (32, 160, 160, 3)
Label: [0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 1 1 0 1 1]


usando o modelo pré treinado no imagenet

In [33]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)

model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

compilando meu modelo

In [34]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

treinando meu modelo

In [36]:
initial_epochs = 4

history = model.fit(ds_train,
                    epochs=initial_epochs,
                    validation_data=ds_val)

Epoch 1/4
582/582 [==============================] - 368s 632ms/step - loss: 0.0832 - accuracy: 0.9685 - val_loss: 0.0790 - val_accuracy: 0.9703
Epoch 2/4
582/582 [==============================] - 397s 683ms/step - loss: 0.0750 - accuracy: 0.9706 - val_loss: 0.0730 - val_accuracy: 0.9718
Epoch 3/4
582/582 [==============================] - 364s 627ms/step - loss: 0.0696 - accuracy: 0.9724 - val_loss: 0.0692 - val_accuracy: 0.9729
Epoch 4/4
582/582 [==============================] - 371s 638ms/step - loss: 0.0656 - accuracy: 0.9744 - val_loss: 0.0662 - val_accuracy: 0.9742


avaliando o modelo

In [37]:
loss, accuracy = model.evaluate(ds_val)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

146/146 [==============================] - 87s 592ms/step - loss: 0.0662 - accuracy: 0.9742
Loss: 0.06617067754268646
Accuracy: 0.9742046594619751
